In [1]:
import streamlit as st
import folium
from streamlit_folium import st_folium
from geopy.geocoders import Nominatim
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import plotly.express as px
from folium import GeoJson, GeoJsonTooltip
import xlrd
pd.set_option('display.max_rows', None)

2025-11-01 22:13:03.660 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:

geolocator = Nominatim(user_agent="Navigator")

@st.cache_data(show_spinner=True, show_time = True)
def geocode_address(address):
    geolocator = Nominatim(user_agent="Navigator")
    return geolocator.geocode(address)

@st.cache_data(show_spinner=True, show_time = True)
def get_osm_features(lat, lon, tags, dist):
    return ox.features_from_point((lat, lon), tags=tags, dist=dist)

@st.cache_data
def load_pie_index(sheet):
    df = pd.read_excel("OSM features.xls", sheet_name=sheet)
    df = df.dropna(subset=["key", "value"])
    df["key"] = df["key"].astype(str).str.strip()
    df["value"] = df["value"].astype(str).str.strip()
    return df

    
def clip_to_circle(gdf, lat, lon, radius):
    if gdf.crs is None:
        gdf = gdf.set_crs(4326)
    proj_crs = gdf.estimate_utm_crs()
    gdf_proj = gdf.to_crs(proj_crs)
    center = Point(lon, lat)
    circle = gpd.GeoSeries([center], crs=4326).to_crs(proj_crs).buffer(radius)
    return gpd.clip(gdf_proj, circle).to_crs(4326)


def melt_tags(gdf, tag_keys):
    # keep onlt keys that exist in gdf
    tag_keys = [k for k in tag_keys if k in gdf.columns]
    if not tag_keys:
        raise ValueError("None of the provided tag_keys exist in the GeoDataFrame.")

    melted = (
        gdf[tag_keys]
        .stack()
        .reset_index()
        .rename(columns={"level_2": "key", 0: "value"})
    )
    melted = melted.merge(gdf.reset_index()[["id", "geometry"]], on="id")
    melted = melted.drop(columns="element")
    melted = gpd.GeoDataFrame(melted, geometry="geometry", crs=gdf.crs)
    return melted

2025-11-01 22:13:07.833 No runtime found, using MemoryCacheStorageManager
2025-11-01 22:13:07.835 No runtime found, using MemoryCacheStorageManager
2025-11-01 22:13:07.838 No runtime found, using MemoryCacheStorageManager


In [3]:
ms_index  = load_pie_index("Multiselect")
ms_index = ms_index[ms_index['Multiselect'].notna()]
ms_index


2025-11-01 22:13:07.852 No runtime found, using MemoryCacheStorageManager
2025-11-01 22:13:07.854 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.270 
  command:

    streamlit run C:\Users\anita\anaconda3\envs\environment\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-01 22:13:08.271 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.272 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.274 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.314 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.316 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.3

,Category,Multiselect,subcategory,key,value,color,icon,description
0,Food & Drinks,bar,substenance,amenity,bar,darkblue,glass,Bar is a purpose-built commercial establishmen...
1,Food & Drinks,cafe,substenance,amenity,cafe,orange,fa-coffee,Cafe is generally an informal place that offer...
2,Food & Drinks,fast food,substenance,amenity,fast_food,orange,cutlery,Fast food restaurant (see also amenity=restaur...
3,Food & Drinks,bar,substenance,amenity,pub,darkblue,fa-beer,A place selling beer and other alcoholic drink...
4,Food & Drinks,restaurant,substenance,amenity,restaurant,orange,cutlery,"Restaurant (not fast food, see amenity=fast_fo..."
5,Education,college,education,amenity,college,lightgrey,education,Campus or buildings of an institute of Further...
6,Education,kindergarten,education,amenity,kindergarten,pink,child,For children too young for a regular school (a...
7,Education,library,education,amenity,library,lightgrey,book,"A public library (municipal, university, …) to..."
8,Education,school,education,amenity,school,lightgrey,education,"School and grounds - primary, middle and secon..."
9,Education,university,education,amenity,university,lightgrey,education,A university campus: an institute of higher ed...


In [4]:
ms_cats = ms_index['Category'].unique()
ms_cats

array(['Food & Drinks', 'Education', 'Transport', 'Services',
       'Healthcare', 'Leisure', 'Stores', 'Religion'], dtype=object)

In [5]:
address = "Skaldevägen 60"
POI_radius=500
location = geocode_address(address)
lat, lon = location.latitude, location.longitude
lat,lon

2025-11-01 22:13:08.372 No runtime found, using MemoryCacheStorageManager
2025-11-01 22:13:08.374 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.375 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.376 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.378 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.599 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.601 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.602 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


(59.3285363, 17.9360367)

In [6]:
poi_tags = {
    "amenity": ["restaurant", "bar", "cafe"],
    "leisure": ["park", "playground"],
    "shop": ["supermarket", "bakery"]
}

In [7]:
ms_poi = get_osm_features(lat, lon, poi_tags, POI_radius)
ms_poi

2025-11-01 22:13:08.632 No runtime found, using MemoryCacheStorageManager
2025-11-01 22:13:08.635 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.636 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.637 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.638 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.815 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.816 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-01 22:13:08.817 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


geometry  \
element  id                                                               
node     248752304                            POINT (17.92894 59.32944)   
         566435738                            POINT (17.92864 59.32886)   
         566435740                            POINT (17.92794 59.32891)   
         1512612813                           POINT (17.92822 59.32942)   
         1516347784                           POINT (17.93867 59.32884)   
         1563539320                            POINT (17.93164 59.3313)   
         4648413053                           POINT (17.93394 59.32674)   
         11975133977                          POINT (17.92904 59.32958)   
         12163401910                          POINT (17.93596 59.32807)   
relation 9680850      POLYGON ((17.93926 59.32994, 17.93969 59.32988...   
way      138575212    POLYGON ((17.92973 59.33108, 17.93072 59.33122...   
         344084533    POLYGON ((17.93969 59.32988, 17.93963 59.33031...   
         587603885    POLYGON ((17.94206 59.32469, 17.94205 59.32483...   
         678517872    POLYGON ((17.93113 59.3317, 17.93119 59.33169,...   
         680473356    POLYGON ((17.93726 59.32442, 17.93734 59.32432...   
         705102013    POLYGON ((17.93324 59.32392, 17.933 59.32385, ...   

                         amenity   cuisine                 name addr:city  \
element  id                                                                 
node     248752304    restaurant     pizza     Nockeby Pizzeria       NaN   
         566435738           NaN       NaN  Coop Engströms Livs    Bromma   
         566435740          cafe       NaN       Nockeby bageri       NaN   
         1512612813   restaurant    indian              Mezbaan       NaN   
         1516347784   restaurant     pizza             Appetito       NaN   
         1563539320   restaurant     pizza  Pizzeria Red Corner       NaN   
         4648413053          NaN       NaN                  NaN       NaN   
         11975133977  restaurant  japanese         PanPan Sushi       NaN   
         12163401910         NaN       NaN           Kea Bageri       NaN   
relation 9680850             NaN       NaN                  NaN       NaN   
way      138575212           NaN       NaN                  NaN       NaN   
         344084533           NaN       NaN                  NaN       NaN   
         587603885           NaN       NaN         Virvelbacken       NaN   
         678517872           NaN       NaN            Brålunden       NaN   
         680473356           NaN       NaN         Thalialunden       NaN   
         705102013           NaN       NaN                  NaN       NaN   

                     addr:postcode   addr:street          alt_name brand  \
element  id                                                                
node     248752304             NaN           NaN               NaN   NaN   
         566435738          167 74  Nockeby Torg  Coop Nockebytorg  Coop   
         566435740             NaN           NaN               NaN   NaN   
         1512612813            NaN           NaN               NaN   NaN   
         1516347784            NaN     Västerled               NaN   NaN   
         1563539320            NaN           NaN               NaN   NaN   
         4648413053            NaN           NaN               NaN   NaN   
         11975133977           NaN           NaN               NaN   NaN   
         12163401910           NaN           NaN               NaN   NaN   
relation 9680850               NaN           NaN               NaN   NaN   
way      138575212             NaN           NaN               NaN   NaN   
         344084533             NaN           NaN               NaN   NaN   
         587603885             NaN           NaN               NaN   NaN   
         678517872             NaN           NaN               NaN   NaN   
         680473356             NaN           NaN               NaN   NaN   
         705102013             Na

In [8]:
poi_data = melt_tags(ms_poi, poi_tags.keys()).reset_index().merge(ms_poi.reset_index()[["id", "name"]], on="id").merge(ms_index[["Category", "Multiselect", "key", "value", "color", "icon"]], on=["key", "value"])
poi_data

,index,id,key,value,geometry,name,Category,Multiselect,color,icon
0,0,248752304,amenity,restaurant,POINT (17.92894 59.32944),Nockeby Pizzeria,Food & Drinks,restaurant,orange,cutlery
1,1,566435738,shop,supermarket,POINT (17.92864 59.32886),Coop Engströms Livs,Stores,supermarket,blue,shopping-cart
2,2,566435740,amenity,cafe,POINT (17.92794 59.32891),Nockeby bageri,Food & Drinks,cafe,orange,fa-coffee
3,3,1512612813,amenity,restaurant,POINT (17.92822 59.32942),Mezbaan,Food & Drinks,restaurant,orange,cutlery
4,4,1516347784,amenity,restaurant,POINT (17.93867 59.32884),Appetito,Food & Drinks,restaurant,orange,cutlery
5,5,1563539320,amenity,restaurant,POINT (17.93164 59.3313),Pizzeria Red Corner,Food & Drinks,restaurant,orange,cutlery
6,6,4648413053,leisure,playground,POINT (17.93394 59.32674),NaN,Leisure,playground,green,fa-child
7,7,11975133977,amenity,restaurant,POINT (17.92904 59.32958),PanPan Sushi,Food & Drinks,restaurant,orange,cutlery
8,8,12163401910,shop,bakery,POINT (17.93596 59.32807),Kea Bageri,Stores,bakery,orange,fa-coffee
9,9,9680850,leisure,park,"POLYGON ((17.93926 59.32994, 17.93969 59.32988...",NaN,Leisure,park,darkgreen,tree-deciduous


In [9]:
poi_data.loc[poi_data['name'].isna(), 'name']="Unnamed"

In [10]:
poi_data

,index,id,key,value,geometry,name,Category,Multiselect,color,icon
0,0,248752304,amenity,restaurant,POINT (17.92894 59.32944),Nockeby Pizzeria,Food & Drinks,restaurant,orange,cutlery
1,1,566435738,shop,supermarket,POINT (17.92864 59.32886),Coop Engströms Livs,Stores,supermarket,blue,shopping-cart
2,2,566435740,amenity,cafe,POINT (17.92794 59.32891),Nockeby bageri,Food & Drinks,cafe,orange,fa-coffee
3,3,1512612813,amenity,restaurant,POINT (17.92822 59.32942),Mezbaan,Food & Drinks,restaurant,orange,cutlery
4,4,1516347784,amenity,restaurant,POINT (17.93867 59.32884),Appetito,Food & Drinks,restaurant,orange,cutlery
5,5,1563539320,amenity,restaurant,POINT (17.93164 59.3313),Pizzeria Red Corner,Food & Drinks,restaurant,orange,cutlery
6,6,4648413053,leisure,playground,POINT (17.93394 59.32674),Unnamed,Leisure,playground,green,fa-child
7,7,11975133977,amenity,restaurant,POINT (17.92904 59.32958),PanPan Sushi,Food & Drinks,restaurant,orange,cutlery
8,8,12163401910,shop,bakery,POINT (17.93596 59.32807),Kea Bageri,Stores,bakery,orange,fa-coffee
9,9,9680850,leisure,park,"POLYGON ((17.93926 59.32994, 17.93969 59.32988...",Unnamed,Leisure,park,darkgreen,tree-deciduous


In [11]:
m = folium.Map(location=[lat, lon], zoom_start=14)         
# Add address marker
folium.Marker([lat, lon], popup=address, icon=folium.Icon(color='red', icon='home')).add_to(m)
folium.Circle(
    location=[lat, lon],
    radius=POI_radius,  # in meters
    color='red',       
    fill=False,
    weight=2            
    ).add_to(m)


In [12]:
poi_layer = folium.FeatureGroup(name="Points of Interest")

for idx, row in poi_data.iterrows():
                    lon_, lat_ = row.geometry.centroid.xy
                    folium.Marker(
                        location=[lat_[0], lon_[0]],
                        popup= f"<div style='font-size:12px; font-family:Arial; white-space:nowrap;'><b>{row.get("Category",'N/A').capitalize()}: </b>{row.get('Multiselect')}<br>{row.get('name', 'Unnamed')}",
                        icon=folium.Icon(
                            color=row['color'],
                            icon=row['icon'].replace("fa-", "") if str(row['icon']).startswith("fa-") else row['icon'],
                            prefix="fa" if str(row['icon']).startswith("fa-") else None
                        )
                        
                        ).add_to(poi_layer)
                         
poi_layer.add_to(m)

In [13]:
m